In [3]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

sns.set(style="whitegrid")


In [4]:
# !pip install xgboost lime

In [5]:
# Load breast cancer dataset
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Filter to "worst" features only
worst_columns = [col for col in df.columns if "worst" in col]
X = df[worst_columns]
y = df["target"]


In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=10000),
    "SVM (RBF)": SVC(kernel="rbf", probability=True),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Train models
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print(f"\n{name} Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(classification_report(y_test, y_pred))



Logistic Regression Accuracy: 0.97
              precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114


SVM (RBF) Accuracy: 0.96
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        43
           1       0.97      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114


Decision Tree Accuracy: 0.94
              precision    recall  f1-score   support

           0       0.91      0.93      0.92        43
           1       0.96      0.94      0.95        71

    accuracy                           0.94       114
   macro avg       0.93      0.94   

/home/vscode/.local/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:29:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [8]:
from sklearn.model_selection import GridSearchCV

# Store best models
best_models = {}

# 1️⃣ Random Forest Grid Search
rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5, scoring='accuracy')
rf_grid.fit(X_train_scaled, y_train)
best_models['Random Forest'] = rf_grid.best_estimator_
print("\n🔍 Best RF Parameters:", rf_grid.best_params_)
print("RF Best CV Score:", rf_grid.best_score_)



🔍 Best RF Parameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}
RF Best CV Score: 0.9604395604395604


In [9]:
# 2️⃣ SVM Grid Search
svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto']
}

svm_grid = GridSearchCV(SVC(probability=True), svm_params, cv=5, scoring='accuracy')
svm_grid.fit(X_train_scaled, y_train)
best_models['SVM'] = svm_grid.best_estimator_
print("\n🔍 Best SVM Parameters:", svm_grid.best_params_)
print("SVM Best CV Score:", svm_grid.best_score_)



🔍 Best SVM Parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
SVM Best CV Score: 0.9714285714285713


In [10]:
# 3️⃣ XGBoost Grid Search
xgb_params = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}

xgb_grid = GridSearchCV(
    xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    xgb_params,
    cv=5,
    scoring='accuracy'
)
xgb_grid.fit(X_train_scaled, y_train)
best_models['XGBoost'] = xgb_grid.best_estimator_
print("\n🔍 Best XGBoost Parameters:", xgb_grid.best_params_)
print("XGBoost Best CV Score:", xgb_grid.best_score_)


/home/vscode/.local/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:30:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:30:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:30:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:30:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.13/site-packages/xgboost/trainin


🔍 Best XGBoost Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
XGBoost Best CV Score: 0.956043956043956


/home/vscode/.local/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:30:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:30:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/vscode/.local/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:30:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [11]:
# Summary of best models and scores
summary = pd.DataFrame({
    model: {
        'Best Score': round(grid.best_score_, 4),
        'Best Params': grid.best_params_
    }
    for model, grid in zip(['Random Forest', 'SVM', 'XGBoost'], [rf_grid, svm_grid, xgb_grid])
}).T

summary


Best Score                                        Best Params
Random Forest     0.9604  {'max_depth': None, 'min_samples_split': 5, 'n...
SVM               0.9714   {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
XGBoost            0.956  {'learning_rate': 0.1, 'max_depth': 3, 'n_esti...

In [30]:
!pip install shap 
!pip uninstall -y numpy
!python3.13 -m pip uninstall -y numpy
!pip install --user numpy==2.2
!pip install numba

import numpy as np
print(np.__version__)

Defaulting to user installation because normal site-packages is not writeable
Found existing installation: numpy 2.2.0
Uninstalling numpy-2.2.0:
  Successfully uninstalled numpy-2.2.0
  Using cached numpy-2.2.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)
Defaulting to user installation because normal site-packages is not writeable
2.3.2


In [31]:
print(np.__version__)
import shap


# Use TreeExplainer for tree-based models
shap.initjs()

# Pick best model from tuning step (e.g., XGBoost)
model = best_models["XGBoost"]

# Refit on full training set if needed
model.fit(X_train_scaled, y_train)

# SHAP Explainer
explainer = shap.Explainer(model)
shap_values = explainer(X_test_scaled)


2.3.2


ImportError: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

In [ ]:
# Summary plot (feature importance across all test samples)
shap.summary_plot(shap_values, features=X_test, feature_names=X.columns)


In [ ]:
# Explain a single prediction
i = 0  # change index to see other examples
shap.plots.waterfall(shap_values[i], max_display=10)


In [ ]:
# Force plot (interactive)
shap.plots.force(shap_values[i])


In [ ]:
import lime
import lime.lime_tabular

# Use best SVM model from tuning step
svm_model = best_models['SVM']
svm_model.fit(X_train_scaled, y_train)

# Create LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_scaled,
    feature_names=X.columns.tolist(),
    class_names=['benign', 'malignant'],
    mode='classification'
)

# Explain a prediction
i = 5  # index of test instance to explain
exp = explainer.explain_instance(
    data_row=X_test_scaled[i],
    predict_fn=svm_model.predict_proba,
    num_features=10
)

# Show explanation
exp.show_in_notebook(show_table=True)


In [ ]:
exp.save_to_file('lime_explanation.html')


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Example: Label Encoding (for binary columns)
def label_encode(df, columns):
    encoder = LabelEncoder()
    for col in columns:
        df[col] = encoder.fit_transform(df[col])
    return df

# Example: One-Hot Encoding (for nominal categories)
def one_hot_encode(df, columns):
    return pd.get_dummies(df, columns=columns, drop_first=True)


In [ ]:
# Example usage (if you had any object columns)
cat_cols = df.select_dtypes(include='object').columns.tolist()
if cat_cols:
    df = one_hot_encode(df, cat_cols)
